In [ ]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import scale

codrna = pd.read_csv("/content/codrna.csv")

In [ ]:
print("Dimensions: ", codrna.shape, "\n")
print(codrna.info())
codrna.head()

Dimensions:  (10000, 8) 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Y       10000 non-null  int64  
 1   X1      10000 non-null  float64
 2   X2      10000 non-null  float64
 3   X3      10000 non-null  float64
 4   X4      10000 non-null  float64
 5   X5      10000 non-null  float64
 6   X6      10000 non-null  float64
 7   X7      10000 non-null  float64
dtypes: float64(7), int64(1)
memory usage: 625.1 KB
None


,Y,X1,X2,X3,X4,X5,X6,X7
0,-1,-4.400806,6.497719,2.487098,6.342305,5.268171,2.817185,6.600208
1,-1,-4.170999,6.650009,2.295134,6.832640,5.147534,2.599744,7.110482
2,-1,-3.722875,6.243902,2.588192,6.938571,4.751345,2.931696,7.220721
3,-1,-4.389316,6.294665,2.424702,6.714761,5.256843,2.746508,6.987810
4,-1,-3.355184,6.599246,2.720926,8.166179,4.149689,3.082046,8.498249


In [ ]:
order = list(np.sort(codrna['Y'].unique()))
print(order)

[-1, 1]


In [ ]:
letter_means = codrna.groupby('Y').mean()
letter_means.head()

,X1,X2,X3,X4,X5,X6,X7
Y,,,,,,,
-1,-2.859452,5.550946,3.952890,4.555901,5.888012,4.484921,4.674731
1,-2.731633,3.964168,3.923074,4.769563,5.876570,4.567878,4.723979


In [ ]:
round(codrna.drop('Y', axis=1).mean(), 2)

X1   -2.79
X2    4.63
X3    3.94
X4    4.68
X5    5.88
X6    4.53
X7    4.70
dtype: float64

In [ ]:
X = codrna.drop("Y", axis = 1)
y = codrna['Y']

In [ ]:
X_scaled = scale(X, with_mean=False)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.3, random_state = 101)

In [ ]:
import random
print("Number of training samples = ", len(X_train))
print("Number of testing samples = ", len(X_test))

Number of training samples =  7000
Number of testing samples =  3000


In [ ]:
# Linear SVM model

model_linear = SVC(kernel='linear')
model_linear.fit(X_train, y_train)
y_pred = model_linear.predict(X_test)

In [ ]:
print("Accuracy linear kernel without Markov Boosting : ", metrics.accuracy_score(y_true=y_test, y_pred=y_pred), "\n")
print(metrics.confusion_matrix(y_true=y_test, y_pred=y_pred))

Accuracy linear kernel without Markov Boosting :  0.9 

[[1007  249]
 [  51 1693]]


In [ ]:
import math
def markov_samp(X_train, Y_train, k = 5, q = 1.2, n2 = 10, T = 10):

#Taking N1 as 5000
    # Dtrain = np.concatenate((X_train, np.array()), axis = 0)
    Dtrain = np.concatenate((np.vstack(X_train), np.vstack(Y_train.to_numpy())), axis = 1)
    Do = random.sample(list(Dtrain), 5000)

    N = len(Do)
    print("N : " , N)

#Initializing mneg and mpos as 0
    t = 1

#Choosing first zt random sample from chosen training samples
    index = np.random.choice(len(Do), 1, replace=False)  
    zsample = np.array(Do)
    # print(type(zsample))
    z = zsample[index][0]

    # print("z : ", z)

#Initializing empty sample holder
    samp = []

#Training SVM for choosing Markov CHain Samples
    model_linear = SVC(kernel='linear')#
    model_linear.fit(X_train, Y_train)#
    
    yzt = model_linear.predict(np.array(z)[0:7].reshape(1, -1))
#Calculating loss for zt sample
    fxy1 = 1
    if not yzt==z[7]:
        fxy1 = 2 
    lzt = math.exp(0-fxy1)

    Pd = 0
    Pdd = 0
    zi = z
#Loop for Markov sampling    
    while(t <= T):
        i = 1
        n1 = 0
        samp = []
        # print(str(t) + " samp " + str(len(samp)) + " N "+ str(N))
        while(i <= N):
            # print("i ", i)
        #Choosing zstar sample 
            zstar = Dtrain[np.random.choice(len(Dtrain), 1, replace=False)][0]
            ystar = model_linear.predict(np.array(zstar)[ 0:7].reshape(1, -1)) #
        #Calculating loss for zstar and P ratio
            fxy = 1
            if not ystar==zstar[7]:
                fxy = 2 
            lzstar = math.exp(0-fxy)

            yzt = model_linear.predict(np.array(zi)[0:7].reshape(1, -1))
            #Calculating loss for zi sample
            fxy1 = 1
            if not yzt==zi[7]:
                fxy1 = 2 
            lzt = math.exp(0-fxy1)

            P = min(1.0, math.exp(lzt-lzstar))

            # print(zt)
            yt = zi[7]
            zi = zstar

            if n1 > n2:
                P = min(1.0, q*P)
                zi = zstar
                Dt = zi
                i = i+1
                n1 = 0

        #Accept/Reject
            rejected = 0
            if P == 1:
                if zi[7] == yt:
                    r = random.uniform(0.001, 1.0)
                    if r <= P:
                        samp.append(zstar) #Pdash = e−y∗f0 /e−ytf0
                        i += 1
                    else:
                        rejected = 1
                else:
                    ct = 1 if yt==1 else -1
                    cst = 1 if ystar==-1 else -1
                    Pd = math.exp(ct*fxy1-cst*fxy)
                    r = random.uniform(0.001, 1.0)
                    if r <= Pd:
                        samp.append(zstar) #P
                        i += 1
                    else:
                        rejected = 1

        #Updating zi          
            zi = zstar
            
        #Checking accept criteria final time and updating n1 if rejection
            if P > 1 or Pd > 1 or Pdd > 1:
                samp.append(zstar)
                i += 1
            if rejected == 1:
                n1 = n1 + 1
            
            # print(len(samp))    
        zi = zstar
        t = t+1

        dtx = np.array(samp)[:, 0:7]
        dty = np.array(samp)[:, 7]

        model_linear = SVC(kernel='linear')#
        model_linear.fit(dtx, dty)#
        y_pred = model_linear.predict(dtx)
        et = 1.0 - metrics.accuracy_score(y_true=dty, y_pred=y_pred)
        if et != 0:
            alpha = 0.5*math.log((1-et)/et)
            if alpha < 0:
                t = t-1
        
    return samp


In [ ]:
nsamp = np.array(markov_samp(X_train, y_train))

X_train = nsamp[:, 0:7]
y_train = nsamp[:, 7]
print("Shape of Markov Sample features : ", X_train.shape)
print("Shape of Markov Sample labels : ", y_train.shape)

N :  5000
Shape of Markov Sample features :  (4996, 7)
Shape of Markov Sample labels :  (4996,)


In [ ]:
# print(y_train)
#Converting label to ASCII in list for sklearn as sampling returned ndarray
Y_train = []
for i in y_train:
    Y_train.append(i)
# print(Y_train)

In [ ]:
print("", X_test.shape)

 (3000, 7)


In [ ]:
# print(y_pred)
#Converting label to ASCII in list for sklearn as training is done on ASCII
Y_test = []
for i in y_test:
    Y_test.append(i)
# print(Y_test)

In [ ]:
# Linear SVM model

model_linear = SVC(kernel='linear')
model_linear.fit(X_train, Y_train)
y_pred = model_linear.predict(X_test)


In [ ]:
# accuracy
print("Accuracy Linear kernel with Markov Boosting : ", metrics.accuracy_score(y_true=Y_test, y_pred=y_pred), "\n")
print(metrics.confusion_matrix(y_true=Y_test, y_pred=y_pred))

Accuracy Linear kernel with Markov Boosting :  0.8996666666666666 

[[1028  228]
 [  73 1671]]
